# Financial Contract Combinators
## A Functional Approach to Modeling Derivatives

Based on Simon Peyton Jones's seminal paper **"Composing Contracts: An Adventure in Financial Engineering"** (2000)

---

### Key Insight

> *"Complex financial derivatives can be elegantly built from simple, composable primitives using functional programming principles."*

This notebook demonstrates how contracts form an **algebra** where sophisticated instruments emerge naturally from fundamental building blocks.

## Part 1: The Primitive Contracts

We start by defining the atomic building blocks from which all derivatives are constructed.

In [ ]:
from dataclasses import dataclass
from typing import Callable, List, Tuple
from enum import Enum
from datetime import datetime, timedelta
import math

class Currency(Enum):
    USD = "USD"
    EUR = "EUR"
    GBP = "GBP"
    JPY = "JPY"

@dataclass
class Observable:
    """Represents an observable value that may vary over time"""
    name: str
    
    def __repr__(self):
        return f"Obs({self.name})"

class Contract:
    """Base class for all contracts"""
    
    def and_contract(self, other: 'Contract') -> 'Contract':
        """Combine two contracts that both execute"""
        return And(self, other)
    
    def or_contract(self, other: 'Contract') -> 'Contract':
        """Choice between two contracts"""
        return Or(self, other)
    
    def __add__(self, other):
        return self.and_contract(other)
    
    def __or__(self, other):
        return self.or_contract(other)

print("✓ Base classes defined")

### The Nine Primitive Combinators

In [ ]:
@dataclass
class Zero(Contract):
    """The worthless contract - neither party gets anything"""
    def __repr__(self):
        return "Zero"

@dataclass
class One(Contract):
    """Immediate payment of one unit of currency"""
    currency: Currency
    def __repr__(self):
        return f"One({self.currency.value})"

@dataclass
class Give(Contract):
    """Reverses the direction of a contract"""
    contract: Contract
    def __repr__(self):
        return f"Give({self.contract})"

@dataclass
class And(Contract):
    """Combination of two contracts, both are acquired"""
    contract1: Contract
    contract2: Contract
    def __repr__(self):
        return f"({self.contract1} & {self.contract2})"

@dataclass
class Or(Contract):
    """Choice between two contracts - holder chooses"""
    contract1: Contract
    contract2: Contract
    def __repr__(self):
        return f"({self.contract1} | {self.contract2})"

@dataclass
class Truncate(Contract):
    """Contract becomes worthless after a certain time"""
    time: int  # days from now
    contract: Contract
    def __repr__(self):
        return f"Truncate({self.time}, {self.contract})"

@dataclass
class Then(Contract):
    """Delays acquisition of a contract"""
    time: int  # days from now
    contract: Contract
    def __repr__(self):
        return f"Then({self.time}, {self.contract})"

@dataclass
class Scale(Contract):
    """Scales the value of a contract by an observable"""
    observable: Observable
    contract: Contract
    def __repr__(self):
        return f"Scale({self.observable}, {self.contract})"

@dataclass
class When(Contract):
    """Delays contract acquisition until an observable becomes True"""
    observable: Observable
    contract: Contract
    def __repr__(self):
        return f"When({self.observable}, {self.contract})"

@dataclass
class Anytime(Contract):
    """Holder may acquire contract at any time before truncation"""
    observable: Observable
    contract: Contract
    def __repr__(self):
        return f"Anytime({self.observable}, {self.contract})"

print("✓ Nine primitive combinators defined")

## Part 2: Building Derivatives from Primitives

Now we construct standard derivatives by composing primitives.

### Example 1: Zero Coupon Bond

A bond that pays a notional amount at maturity.

In [ ]:
def zcb(maturity: int, notional: float, currency: Currency) -> Contract:
    """Zero Coupon Bond - receive notional at maturity"""
    return Then(maturity, Scale(Observable(str(notional)), One(currency)))

# Create a 1-year bond paying $1000
bond = zcb(365, 1000, Currency.USD)
print("Zero Coupon Bond:")
print(f"  {bond}")
print("\nInterpretation: In 365 days, scale One(USD) by 1000")

### Example 2: European Call Option

The right (but not obligation) to buy an asset at a strike price.

In [ ]:
def european_call(strike: float, maturity: int, underlying: str, currency: Currency) -> Contract:
    """European call option"""
    payoff_obs = Observable(f"max(0, {underlying} - {strike})")
    return Then(maturity, Scale(payoff_obs, One(currency)))

# Create a call option
call = european_call(100, 90, "AAPL", Currency.USD)
print("European Call Option (Strike=$100, 90 days, on AAPL):")
print(f"  {call}")
print("\nPayoff: max(0, AAPL - 100) at day 90")

### Example 3: European Put Option

In [ ]:
def european_put(strike: float, maturity: int, underlying: str, currency: Currency) -> Contract:
    """European put option"""
    payoff_obs = Observable(f"max(0, {strike} - {underlying})")
    return Then(maturity, Scale(payoff_obs, One(currency)))

put = european_put(100, 90, "AAPL", Currency.USD)
print("European Put Option (Strike=$100, 90 days, on AAPL):")
print(f"  {put}")
print("\nPayoff: max(0, 100 - AAPL) at day 90")

## Part 3: Compositional Power - Building Complex from Simple

The real magic: complex strategies emerge from combining simple contracts.

### Straddle (Long Call + Long Put)

In [ ]:
# Combine call and put at same strike
straddle = call + put

print("Straddle Strategy:")
print(f"  {straddle}")
print("\nProfit from large moves in either direction!")

### Bull Call Spread (Long Low Strike, Short High Strike)

In [ ]:
long_call = european_call(100, 90, "AAPL", Currency.USD)
short_call = Give(european_call(110, 90, "AAPL", Currency.USD))
bull_spread = long_call + short_call

print("Bull Call Spread:")
print(f"  Long call at $100")
print(f"  Short call at $110")
print(f"\nStructure: {bull_spread}")
print("\nLimited profit above $110, limited loss below $100")

### American Option (Early Exercise Allowed)

In [ ]:
def american_call(strike: float, maturity: int, underlying: str, currency: Currency) -> Contract:
    """American call option - can exercise anytime before maturity"""
    payoff_obs = Observable(f"max(0, {underlying} - {strike})")
    exercise_condition = Observable(f"{underlying} > {strike}")
    return Truncate(maturity, 
                    Anytime(exercise_condition, 
                           Scale(payoff_obs, One(currency))))

american = american_call(100, 90, "AAPL", Currency.USD)
print("American Call Option:")
print(f"  {american}")
print("\nCan exercise anytime when AAPL > 100, expires in 90 days")

## Part 4: Algebraic Properties

Contracts satisfy mathematical laws that enable reasoning about equivalences.

### Put-Call Parity

**Mathematical fact**: Call - Put = Forward

This relationship emerges naturally from the combinator structure!

In [ ]:
# Synthetic forward: Long call + Short put
call = european_call(100, 90, "S", Currency.USD)
short_put = Give(european_put(100, 90, "S", Currency.USD))
synthetic_forward = call + short_put

print("Put-Call Parity:")
print("  Call - Put = Forward")
print("\nSynthetic Forward (Long Call + Short Put):")
print(f"  {synthetic_forward}")
print("\nThis has the same payoff profile as a forward contract!")

# Actual forward contract
def forward_contract(strike: float, maturity: int, underlying: str, currency: Currency) -> Contract:
    payoff_obs = Observable(f"{underlying} - {strike}")
    return Then(maturity, Scale(payoff_obs, One(currency)))

forward = forward_contract(100, 90, "S", Currency.USD)
print("\nDirect Forward Contract:")
print(f"  {forward}")
print("\n✓ No-arbitrage: Both must have the same value!")

### Other Algebraic Laws

1. **Zero is identity**: `Zero & c = c`
2. **Give is involutive**: `Give(Give(c)) = c`
3. **Commutativity**: `c1 & c2 = c2 & c1`
4. **Associativity**: `(c1 & c2) & c3 = c1 & (c2 & c3)`

## Part 5: Contract Valuation

Now let's add pricing capabilities to our contracts.

In [ ]:
from typing import Dict

class PricingModel:
    """Simple pricing model for demonstrating contract valuation"""
    
    def __init__(self, spot_prices: Dict[str, float], 
                 risk_free_rate: float,
                 volatilities: Dict[str, float]):
        self.spot_prices = spot_prices
        self.risk_free_rate = risk_free_rate
        self.volatilities = volatilities
        
    def black_scholes_call(self, S: float, K: float, T: float, 
                          r: float, sigma: float) -> float:
        """Black-Scholes formula for European call"""
        if T <= 0:
            return max(S - K, 0)
        
        d1 = (math.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*math.sqrt(T))
        d2 = d1 - sigma*math.sqrt(T)
        
        N = lambda x: 0.5 * (1 + math.erf(x/math.sqrt(2)))
        
        return S * N(d1) - K * math.exp(-r*T) * N(d2)
    
    def black_scholes_put(self, S: float, K: float, T: float,
                         r: float, sigma: float) -> float:
        """Black-Scholes formula for European put"""
        if T <= 0:
            return max(K - S, 0)
            
        d1 = (math.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*math.sqrt(T))
        d2 = d1 - sigma*math.sqrt(T)
        
        N = lambda x: 0.5 * (1 + math.erf(x/math.sqrt(2)))
        
        return K * math.exp(-r*T) * N(-d2) - S * N(-d1)

print("✓ Pricing model defined")

### Value Some Contracts

In [ ]:
# Setup market data
model = PricingModel(
    spot_prices={'AAPL': 150.0, 'S': 100.0},
    risk_free_rate=0.05,
    volatilities={'AAPL': 0.25, 'S': 0.25}
)

print("Market Data:")
print(f"  AAPL spot: ${model.spot_prices['AAPL']}")
print(f"  Risk-free rate: {model.risk_free_rate*100}%")
print(f"  Volatility: {model.volatilities['AAPL']*100}%")
print()

# Price some options
T = 90/365.0

call_value = model.black_scholes_call(150, 100, T, 0.05, 0.25)
print(f"Call Option (K=100, S=150, T=90d): ${call_value:.2f}")
print(f"  Intrinsic value: ${max(150-100, 0):.2f}")
print(f"  Time value: ${call_value - max(150-100, 0):.2f}")
print()

put_value = model.black_scholes_put(150, 100, T, 0.05, 0.25)
print(f"Put Option (K=100, S=150, T=90d): ${put_value:.2f}")
print(f"  Out-of-the-money, all time value")

### Verify Put-Call Parity Numerically

In [ ]:
S = 100
K = 100
T = 90/365.0
r = 0.05
sigma = 0.25

call_val = model.black_scholes_call(S, K, T, r, sigma)
put_val = model.black_scholes_put(S, K, T, r, sigma)

# Put-Call Parity: C - P = S - K*exp(-rT)
lhs = call_val - put_val
rhs = S - K * math.exp(-r * T)

print("Put-Call Parity Verification:")
print(f"  Call value: ${call_val:.4f}")
print(f"  Put value: ${put_val:.4f}")
print()
print(f"  C - P = ${lhs:.4f}")
print(f"  S - PV(K) = ${rhs:.4f}")
print(f"  Difference: ${abs(lhs - rhs):.8f}")
print()
if abs(lhs - rhs) < 0.0001:
    print("✓ Put-Call Parity VERIFIED!")
else:
    print("✗ Parity violation")

## Part 6: Exotic Derivatives

The framework naturally extends to exotic derivatives.

### Digital (Binary) Option

Fixed payout if condition is met, nothing otherwise.

In [ ]:
def digital_call(strike: float, maturity: int, payout: float,
                underlying: str, currency: Currency) -> Contract:
    """Digital/Binary call - fixed payout if above strike"""
    indicator = Observable(f"1 if {underlying} > {strike} else 0")
    return Then(maturity, Scale(Observable(str(payout)), 
                               Scale(indicator, One(currency))))

digital = digital_call(100, 90, 100, "AAPL", Currency.USD)
print("Digital Call Option:")
print(f"  {digital}")
print("\nPayoff: $100 if AAPL > 100, else $0")

### Rainbow Option (Best-of Multiple Assets)

In [ ]:
def rainbow_option(strike: float, maturity: int, 
                   underlyings: List[str], option_type: str,
                   currency: Currency) -> Contract:
    """Rainbow option - payoff on best/worst of multiple assets"""
    if option_type == "best-of":
        payoff = Observable(f"max(0, max([{', '.join(underlyings)}]) - {strike})")
    else:  # worst-of
        payoff = Observable(f"max(0, min([{', '.join(underlyings)}]) - {strike})")
    return Then(maturity, Scale(payoff, One(currency)))

rainbow = rainbow_option(100, 90, ["AAPL", "GOOGL", "MSFT"], 
                        "best-of", Currency.USD)
print("Rainbow Option (Best-of-3):")
print(f"  {rainbow}")
print("\nPayoff based on the best performer among 3 stocks")

### Barrier Option (Knock-In)

In [ ]:
def barrier_option(strike: float, barrier: float, maturity: int, 
                   underlying: str, barrier_type: str, currency: Currency) -> Contract:
    """Knock-in or knock-out barrier option"""
    payoff_obs = Observable(f"max(0, {underlying} - {strike})")
    
    if barrier_type == "knock-in":
        barrier_condition = Observable(f"{underlying} > {barrier}")
        return Truncate(maturity,
                       When(barrier_condition,
                            Then(maturity, Scale(payoff_obs, One(currency)))))

barrier = barrier_option(100, 120, 90, "AAPL", "knock-in", Currency.USD)
print("Barrier Option (Knock-In):")
print(f"  {barrier}")
print("\nActivates only if AAPL touches $120 before maturity")

## Part 7: Structured Products

Complex retail products built from primitives.

### Principal Protected Note

100% principal protection + participation in equity upside

In [ ]:
# Zero coupon bond for principal protection
bond = zcb(365, 100, Currency.USD)

# Call option for upside participation (80% participation)
call = european_call(100, 365, "SPX", Currency.USD)
scaled_call = Scale(Observable("0.8"), call)

# Combine them
ppn = bond + scaled_call

print("Principal Protected Note:")
print("  Structure: 100% principal + 80% equity upside")
print()
print("  Bond component ensures principal return")
print("  Call option provides market participation")
print()
print("Worst case: Return of $100 principal")
print("Best case: $100 + 80% of SPX gains above 100")

### Reverse Convertible

High coupon note with downside risk

In [ ]:
# High quarterly coupons
coupon_dates = [90, 180, 270, 365]
coupons = Zero()
for date in coupon_dates:
    coupon = Then(date, Scale(Observable("2.5"), One(Currency.USD)))
    coupons = coupons + coupon

# Principal repayment
principal = Then(365, Scale(Observable("100"), One(Currency.USD)))

# Short put - THE RISK
short_put = Give(european_put(80, 365, "AAPL", Currency.USD))

# Combine
reverse_conv = coupons + principal + short_put

print("Reverse Convertible:")
print("  High coupon: 10% annual (2.5% quarterly)")
print("  Principal: $100")
print("  Risk: Short put at $80")
print()
print("If AAPL stays above $80: Collect high coupons + principal")
print("If AAPL falls below $80: Receive shares instead of cash")

## Part 8: Real-World Trading Strategies

### Iron Condor

Profit from low volatility (range-bound market)

In [ ]:
# Put spread
short_put = Give(european_put(95, 90, "AAPL", Currency.USD))
long_put = european_put(90, 90, "AAPL", Currency.USD)

# Call spread
short_call = Give(european_call(105, 90, "AAPL", Currency.USD))
long_call = european_call(110, 90, "AAPL", Currency.USD)

# Combine all four
iron_condor = short_put + long_put + short_call + long_call

print("Iron Condor:")
print("  Short put @ 95")
print("  Long put @ 90")
print("  Short call @ 105")
print("  Long call @ 110")
print()
print("Profit zone: $95 - $105")
print("Max profit: Net premium received")
print("Max loss: Width of spread - premium")

### Collar Strategy

Protective put + covered call (limits both gains and losses)

In [ ]:
# Own the stock
stock = Then(0, Scale(Observable("AAPL"), One(Currency.USD)))

# Buy protective put
protective_put = european_put(90, 365, "AAPL", Currency.USD)

# Sell covered call
covered_call = Give(european_call(110, 365, "AAPL", Currency.USD))

# Combine
collar = stock + protective_put + covered_call

print("Collar Strategy:")
print("  Long stock")
print("  Long put @ $90 (downside protection)")
print("  Short call @ $110 (cap on upside)")
print()
print("Result: Limited profit ($110), limited loss ($90)")
print("Often zero-cost if call premium offsets put cost")

## Part 9: Key Insights and Conclusions

### Mathematical Elegance

1. **Compositionality**: Complex derivatives = composition of simple primitives
2. **Linearity**: Valuation preserves addition: `V(c1 + c2) = V(c1) + V(c2)`
3. **No Arbitrage**: Mathematical relationships (like put-call parity) emerge naturally
4. **Transparency**: Every component is explicit, no "black boxes"
5. **Type Safety**: Contract structure prevents many modeling errors

### Practical Benefits

1. **Reusability**: Building blocks work across all products
2. **Testing**: Individual primitives tested in isolation
3. **Risk Management**: Clear decomposition of portfolio exposures
4. **Communication**: Transparent explanation of structured products
5. **Model Validation**: Check pricing consistency algebraically

### The Power of Functional Programming

> *"The great power of functional programming is that it allows us to build complex systems from simple, well-understood components through composition."* - Simon Peyton Jones

This approach provides:
- **Modularity** without sacrificing expressiveness
- **Correctness** without sacrificing flexibility
- **Abstraction** without sacrificing performance
- **Simplicity** without sacrificing power

## Summary: Contract Algebra

Contracts form a **commutative semiring**:

| Operation | Combinator | Properties |
|-----------|------------|------------|
| Zero element | `Zero` | `Zero & c = c` |
| Addition | `And` | Commutative, Associative |
| Reversal | `Give` | Involutive: `Give(Give(c)) = c` |
| Delay | `Then` | Time value: `V(Then(t,c)) = e^(-rt)·V(c)` |
| Scaling | `Scale` | Linear: `V(Scale(k,c)) = k·V(c)` |
| Choice | `Or` | Max value: `V(c1 | c2) = max(V(c1), V(c2))` |

These primitives + composition = **ALL** derivative contracts!

## Further Exploration

Try modifying the examples:
1. Create your own exotic derivative
2. Build a custom structured product
3. Implement multi-currency contracts
4. Add credit risk (default observables)
5. Model collateralization agreements

The possibilities are endless!

## References

1. **Peyton Jones, S., Eber, J-M., & Seward, J.** (2000). "Composing Contracts: An Adventure in Financial Engineering." *ICFP 2000*.

2. **Eber, J-M.** (1999). "Compositional Specification of Financial Contracts." *PhD Thesis, École Polytechnique*.

3. **Hull, J.C.** (2017). *Options, Futures, and Other Derivatives*. Pearson.

---

**End of Notebook**